In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [338]:
# read data from file
train = pd.read_csv("./input/train.csv") 
test = pd.read_csv("./input/test.csv")
#print(train['Rating'])
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [0]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [0]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [0]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [342]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [0]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [0]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [345]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [0]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [347]:
train = data_clean_3(train)
test = data_clean_3(test)
print(test.columns)
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Index(['Brand', 'SIM 2', 'RAM', 'os_name', 'SIM Slot(s)', 'Height',
       'Pixel Density', 'Processor_frequency',
       'Screen to Body Ratio (calculated)', 'Capacity', 'PhoneId',
       'Resolution', 'Sim1', 'Num_cores', 'Internal Memory', 'Screen Size',
       'Weight'],
      dtype='object')
Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [348]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
print(cols)
cols.remove('PhoneId')
print(cols)
cols.insert(0, 'PhoneId')
print("Train columns are",train.columns)
#combined=train.drop('Rating', axis=1)[cols]
combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

['Brand', 'SIM 2', 'RAM', 'os_name', 'SIM Slot(s)', 'Height', 'Pixel Density', 'Processor_frequency', 'Screen to Body Ratio (calculated)', 'Capacity', 'PhoneId', 'Resolution', 'Sim1', 'Num_cores', 'Internal Memory', 'Screen Size', 'Weight']
['Brand', 'SIM 2', 'RAM', 'os_name', 'SIM Slot(s)', 'Height', 'Pixel Density', 'Processor_frequency', 'Screen to Body Ratio (calculated)', 'Capacity', 'Resolution', 'Sim1', 'Num_cores', 'Internal Memory', 'Screen Size', 'Weight']
Train columns are Index(['Brand', 'SIM 2', 'RAM', 'os_name', 'SIM Slot(s)', 'Height',
       'Pixel Density', 'Processor_frequency',
       'Screen to Body Ratio (calculated)', 'Capacity', 'PhoneId',
       'Resolution', 'Sim1', 'Num_cores', 'Internal Memory', 'Screen Size',
       'Rating', 'Weight'],
      dtype='object')
(460, 17)
Index(['PhoneId', 'Brand', 'SIM 2', 'RAM', 'os_name', 'SIM Slot(s)', 'Height',
       'Pixel Density', 'Processor_frequency',
       'Screen to Body Ratio (calculated)', 'Capacity', 'Resolution

In [0]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [350]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [351]:
#list(set(train_new.columns) - set(test_new.columns))
#print(test_new.columns)
train_new.head()

,PhoneId,RAM,Height,Pixel Density,Processor_frequency,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Screen Size,...,Sim1_4G,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,Rating
0,0,4,157.9,403,1.8,80.68,4000,20,64,6.26,...,1,0,0,0,0,1,0,0,0,4.5
1,1,3,156.2,271,1.8,80.85,4230,8,32,6.20,...,1,0,0,0,0,1,0,0,0,4.5
2,2,3,157.0,409,2.1,83.68,3500,25,32,6.30,...,1,0,0,0,0,1,0,0,0,4.4
3,4,4,159.8,411,2.2,74.78,3300,24,64,6.00,...,1,0,0,0,0,1,0,0,0,4.3
4,5,4,160.4,396,2.2,84.23,3750,16,64,6.50,...,1,0,0,0,0,1,0,0,0,4.4


In [352]:
test_new.head()

,PhoneId,RAM,Height,Pixel Density,Processor_frequency,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Screen Size,...,Sim1_3G,Sim1_4G,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa
0,3,2,156.2,271,1.8,80.85,4230,5,16,6.20,...,0,1,0,0,0,0,1,0,0,0
1,11,4,156.0,402,1.8,81.60,5000,12,64,6.20,...,0,1,0,0,0,0,1,0,0,0
2,13,6,156.7,409,2.0,83.84,3500,25,64,6.30,...,0,1,0,0,0,0,1,0,0,0
3,16,4,156.7,409,2.0,83.84,3500,16,64,6.30,...,0,1,0,0,0,0,1,0,0,0
4,19,4,158.6,403,1.8,77.43,4000,20,64,5.99,...,0,1,0,0,0,0,1,0,0,0


## Solution

In [0]:
#train_new=train_new.drop(train_new[train_new.Rating == 4 ].index)
#train_new=train_new.drop(train_new[i for i in train_new.Rating if i >= 3.5 and i < 4].index)
train_new['Rating']=train_new['Rating'].map(lambda x : 1 if x >=4  else 0)
#train_new['Rating']=train_new['Rating'].map(lambda x : 1 if x > 4  else 0)
train_new=train_new.drop('PhoneId',axis=1)

## Variance Calculation

In [354]:
normalise=pd.DataFrame(columns=train_new.columns)
for i in train_new.columns:
  if train_new[i].min()==train_new[i].max():
    normalise[i]=0
  else:
    normalise[i]=(train_new[i] - train_new[i].min())/(train_new[i].max()-train_new[i].min())
#print(normalise)
variance=normalise.var(axis=0)
variance=pd.DataFrame(variance)
variance=variance.T
#normalise.head()
variance.head()
avg_variance=variance.mean(axis=1)
#print(avg_variance)
drop_columns=[]
for x in variance.columns:
  if variance[x][0]==0:
    drop_columns.append(x)
#print(train_new.shape[1])

                                                RAM    Height  Pixel Density  \
RAM                                        1.000000 -0.227798      -0.209022   
Height                                    -0.227798  1.000000       0.440141   
Pixel Density                             -0.209022  0.440141       1.000000   
Processor_frequency                       -0.155368  0.373361       0.639060   
Screen to Body Ratio (calculated)         -0.328073  0.744119       0.480382   
Capacity                                  -0.197554  0.635365       0.296012   
Resolution                                -0.074343  0.469851       0.282635   
Internal Memory                           -0.098095  0.356375       0.428678   
Screen Size                               -0.405488  0.861143       0.484592   
Weight                                    -0.249511  0.642595       0.384774   
Brand_10.or                               -0.010813 -0.018798      -0.042938   
Brand_Apple                             

## Delete the columns with less variance from the average variance

In [355]:
for i in drop_columns:
  train_new=train_new.drop(i,axis=1)
  test_new=test_new.drop(i,axis=1)
print(train_new.shape)
train_new.head()


(341, 84)


,RAM,Height,Pixel Density,Processor_frequency,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Screen Size,Weight,...,Sim1_4G,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,Rating
0,4,157.9,403,1.8,80.68,4000,20,64,6.26,182,...,1,0,0,0,0,1,0,0,0,1
1,3,156.2,271,1.8,80.85,4230,8,32,6.20,168,...,1,0,0,0,0,1,0,0,0,1
2,3,157.0,409,2.1,83.68,3500,25,32,6.30,168,...,1,0,0,0,0,1,0,0,0,1
3,4,159.8,411,2.2,74.78,3300,24,64,6.00,169,...,1,0,0,0,0,1,0,0,0,1
4,4,160.4,396,2.2,84.23,3750,16,64,6.50,175,...,1,0,0,0,0,1,0,0,0,1


## Threshold for Binarization

In [356]:
train_mean=train_new.groupby('Rating').mean()
print(train_mean)
train_new.head()
Y=train_new['Rating']
X=train_new.drop('Rating',axis=1)

print("X=",X.head())

phone_id=test_new['PhoneId']
test_new=test_new.drop('PhoneId',axis=1)
phone_id=pd.DataFrame(phone_id)
Y=pd.DataFrame(Y)

index=['thres','value']
columns=X.columns
mean_data=pd.DataFrame(index=index,columns=columns)

for i in X.columns:
  mean_data[i]['thres']=(train_mean[i][0]+train_mean[i][1])/2
  if train_mean[i][0]>train_mean[i][1]:
    mean_data[i]['value']=0
  else:
    mean_data[i]['value']=1
print(X.columns)

              RAM      Height  Pixel Density  Processor_frequency  \
Rating                                                              
0       27.543689  146.980823     327.116505             1.568447   
1        4.012605  151.229813     351.609244             1.889538   

        Screen to Body Ratio (calculated)     Capacity  Resolution  \
Rating                                                               
0                               69.088544  2984.805825    7.601942   
1                               73.766261  3280.676471   10.453782   

        Internal Memory  Screen Size     Weight         ...           Sim1_3G  \
Rating                                                  ...                     
0             30.330097     5.163689  158.84466         ...          0.048544   
1             53.310924     5.591555  162.42437         ...          0.008403   

         Sim1_4G  Num_cores_312  Num_cores_Deca  Num_cores_Dual  \
Rating                                            

## Binarization of Input

In [357]:
for i in X.columns:
  X[i]=X[i].map(lambda x : mean_data[i]['value'] if x>=mean_data[i]['thres'] else int(not mean_data[i]['value']))
  
for j in test_new.columns:
  test_new[j]=test_new[j].map(lambda x : mean_data[j]['value'] if x>=mean_data[j]['thres'] else int(not mean_data[j]['value']))

print(type(X))
print(test_new)

<class 'pandas.core.frame.DataFrame'>
     RAM  Height  Pixel Density  Processor_frequency  \
0      1       1              0                    1   
1      1       1              1                    1   
2      1       1              1                    1   
3      1       1              1                    1   
4      1       1              1                    1   
5      1       1              1                    1   
6      1       1              1                    1   
7      1       0              0                    1   
8      1       1              0                    1   
9      1       1              0                    0   
10     1       1              1                    1   
11     1       1              0                    1   
12     1       0              0                    0   
13     1       1              0                    1   
14     1       0              1                    1   
15     1       1              1                    1   
16     1  

## Train - Test Split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1, stratify=Y, random_state=1)

## MP Neuron

In [0]:
class MPNeuron:
  
  
  def __init__(self):
    self.b=None
    
  def model(self,X):
    return (1 if np.sum(X)>=self.b else 0)
  
  def predict(self,X):
    y_pred=[]
    for x in X:
      y_pred.append(self.model(x))
    return np.asarray(y_pred)
    
  def fit(self,X,Y):
    accuracy={}
    
    for b in range(X.shape[1]+1):
      self.b=b
      y_pred=self.predict(X)
      accuracy[b]=accuracy_score(y_pred,Y)
    best_b=max(accuracy,key=accuracy.get)
    print(accuracy)
    self.b=best_b
    print("The best bias is",self.b)
    print("The accuray of the bias is",accuracy[self.b])
    

In [360]:
print(Y_train.head())
X_train=X_train.values
Y_train=Y_train.values
#print(Y_train)
X_test=X_test.values
Y_test=Y_test.values
test_data=test_new.values
mp_neuron=MPNeuron()
mp_neuron.fit(X_train,Y_train)
test_pred=mp_neuron.predict(X_test)
pred_out=mp_neuron.predict(test_data)
print(pred_out)
pred_out=pd.DataFrame(pred_out)
pred_out.columns=['Class']
test_new=pd.DataFrame(test_new)
#pred_out=pred_out.values
#print(pred_out)
print(type(test_new))
print(accuracy_score(test_pred,Y_test))

     Rating
248       0
58        1
24        1
200       1
182       0
{0: 0.6993464052287581, 1: 0.6993464052287581, 2: 0.6993464052287581, 3: 0.6993464052287581, 4: 0.6993464052287581, 5: 0.6993464052287581, 6: 0.6993464052287581, 7: 0.6993464052287581, 8: 0.6993464052287581, 9: 0.6993464052287581, 10: 0.6993464052287581, 11: 0.6993464052287581, 12: 0.6993464052287581, 13: 0.6993464052287581, 14: 0.6993464052287581, 15: 0.6993464052287581, 16: 0.6993464052287581, 17: 0.6993464052287581, 18: 0.6993464052287581, 19: 0.6993464052287581, 20: 0.6993464052287581, 21: 0.6993464052287581, 22: 0.6993464052287581, 23: 0.6993464052287581, 24: 0.6993464052287581, 25: 0.6993464052287581, 26: 0.6993464052287581, 27: 0.6993464052287581, 28: 0.6993464052287581, 29: 0.6993464052287581, 30: 0.6993464052287581, 31: 0.6993464052287581, 32: 0.6993464052287581, 33: 0.6993464052287581, 34: 0.7058823529411765, 35: 0.7156862745098039, 36: 0.7222222222222222, 37: 0.7222222222222222, 38: 0.7156862745098039, 3

In [361]:
submission=pd.DataFrame({'PhoneId':phone_id['PhoneId'], 'Class':pred_out['Class']})
submission = submission[['PhoneId', 'Class']]
print(submission.head())
submission.to_csv("submission.csv", index=False)
print(submission.shape)

   PhoneId  Class
0        3      1
1       11      1
2       13      1
3       16      1
4       19      1
(119, 2)


In [362]:
"""
WRITE YOUR MODELLING CODE HERE
"""

'\nWRITE YOUR MODELLING CODE HERE\n'